# Import Packages


In [ ]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import skimage
from skimage.transform import resize
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import os

print("Packages imported successfully")

# Drive Mounting

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Import Training Data

In [ ]:
training_dir = '/content/drive/MyDrive/VisionProject/Dataset/ASL_Alphabet/asl_alphabet_train/asl_alphabet_train'

def import_train_data(folder):
  x = []
  y = []
  cont = 0
  
  mask = {
      "A": 0,
      "B": 1,
      "C": 2,
      "D": 3,
      "E": 4,
      "F": 5,
      "G": 6,
      "H": 7,
      "I": 8,
      "J": 9,
      "K": 10,
      "L": 11,
      "M": 12,
      "N": 13,
      "O": 14,
      "P": 15,
      "Q": 16,
      "R": 17,
      "S": 18,
      "T": 19,
      "U": 20,
      "V": 21,
      "W": 22,
      "X": 23,
      "Y": 24,
      "Z": 25,
      "del": 26,
      "nothing": 27,
      "space": 28,
  }
  for folderName in os.listdir(folder):
    cont_per_class = 0
    if not folderName.startswith('.'):
      label = mask[folderName]
      for image_filename in os.listdir(folder + '/' + folderName):
        img_file = cv2.imread(folder + '/' + folderName + '/' + image_filename)
        if img_file is not None:
          img_file = skimage.transform.resize(img_file, (64, 64, 3))
          img_arr = np.asarray(img_file)
              
          x.append(img_arr)
          y.append(label)
          cont += 1
          # if cont > 2000:
          #   return np.array(x), np.array(y)
          cont_per_class += 1
        if cont_per_class == 1700:
            print("Class " + folderName + " done")
            break

  return np.array(x), np.array(y)

x_train, y_train = import_train_data(training_dir)

print("Training images successfully imported")



# Import Test Data

In [ ]:
test_dir = '/content/drive/MyDrive/VisionProject/Dataset/ASL_Alphabet/asl_alphabet_test/asl_alphabet_test'

def import_test_data(folder):
  x = []
  y = []
  cont = 0 
  mask = {
      "A": 0,
      "B": 1,
      "C": 2,
      "D": 3,
      "E": 4,
      "F": 5,
      "G": 6,
      "H": 7,
      "I": 8,
      "J": 9,
      "K": 10,
      "L": 11,
      "M": 12,
      "N": 13,
      "O": 14,
      "P": 15,
      "Q": 16,
      "R": 17,
      "S": 18,
      "T": 19,
      "U": 20,
      "V": 21,
      "W": 22,
      "X": 23,
      "Y": 24,
      "Z": 25,
      "del": 26,
      "nothing": 27,
      "space": 28,
  }
  for folderName in os.listdir(folder):
    cont_per_class = 0
    if not folderName.startswith('.'):
      label = mask[folderName]
      for image_filename in os.listdir(folder + '/' + folderName):
        img_file = cv2.imread(folder + '/' + folderName + '/' + image_filename)
        if img_file is not None:
          img_file = skimage.transform.resize(img_file, (64, 64, 3))
          img_arr = np.asarray(img_file)
              
          x.append(img_arr)
          y.append(label)
          cont += 1
          # if cont > 1000:
          #   return np.array(x), np.array(y)
          cont_per_class += 1
        if cont_per_class == 550:
          print("Class " + folderName + " done")
          break
  return np.array(x), np.array(y)

x_test, y_test = import_test_data(test_dir)

print("Test images successfully imported")


# Shape Check

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

#One-Hot Encoding

In [ ]:
y_ohe_train = to_categorical(y_train, 29)
print(y_ohe_train.shape)
print(y_ohe_train[0])
y_ohe_test = to_categorical(y_test, 29)
print(y_ohe_test.shape)
print(y_ohe_test[0])

# Training Data Split

In [7]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_ohe_train, test_size=0.3,random_state=42,shuffle = True)

# Model Architecture

In [ ]:
model = Sequential()

model.add(Conv2D(32, (5,5), input_shape = (64, 64, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization(momentum=0.99))
model.add(MaxPooling2D((2,2)))
# model.add(Dropout(0.7))


model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization(momentum=0.99))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.7))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization(momentum=0.99))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.7))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dense(29, activation = 'softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

print(model.summary())


# Model 1 Architecture

In [ ]:
model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(29, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

print(model.summary())

# Model 2 Architecture

In [ ]:

  model = Sequential()
    
  model.add(Conv2D(64, (3, 3),padding='same',input_shape=(64, 64, 3)))
  model.add(Conv2D(64, (3, 3),padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3),padding='same',input_shape=(64, 64, 3)))
  model.add(Conv2D(64, (3, 3),padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3),padding='same',input_shape=(64, 64, 3)))
  model.add(Conv2D(64, (3, 3),padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(128))
  model.add(Dropout(0.5))

  model.add(Dense(128))
  model.add(Dropout(0.5))

  model.add(Dense(29))
  model.add(Activation('softmax'))
  
  opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
  model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

  print(model.summary())

# Training

In [ ]:
early_stops = EarlyStopping(monitor = 'val_loss', patience = 5)
checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/VisionProject/Weights/weights_best.hdf5', verbose=1, save_best_only=True) # Model
#checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/VisionProject/Weights/weights_best1.hdf5', verbose=1, save_best_only=True) # Model 1
#checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/VisionProject/Weights/weights_best2.hdf5', verbose=1, save_best_only=True) # Model 2
history = model.fit(x_train, y_train,
          epochs = 10,
          batch_size = 64,
          verbose = 2,
          validation_data = (x_valid, y_valid),
          callbacks = [early_stops, checkpointer])

In [11]:
def plot_history(history,name="Model"):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(name + ' accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('/content/drive/MyDrive/VisionProject/Results/Accuracy/accuracy1700-550(Final 3 - 14 - check[20 epoche][tutte]BatchNormalization=0.99&[2 e 3]drop=0.7&opt).png') # Model
    #plt.savefig('/content/drive/MyDrive/VisionProject/Results/Accuracy/accuracy1700-550(Final Model 1 - 8 - check[20 epoche].png') # Model 1
    #plt.savefig('/content/drive/MyDrive/VisionProject/Results/Accuracy/accuracy1700-550(Final Model 2 - 2 - check[20 epoche].png') # Model 2
    plt.show()
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(name + ' loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('/content/drive/MyDrive/VisionProject/Results/Loss/loss1700-550(Final 3 - 14 - check[10 epoche][tutte]BatchNormalization=0.99&[2 e 3]drop=0.7&opt).png') # Model
    #plt.savefig('/content/drive/MyDrive/VisionProject/Results/Loss/Loss1700-550(Final Model 1 - 8 - check[10 epoche].png') # Model 1
    #plt.savefig('/content/drive/MyDrive/VisionProject/Results/Loss/Loss1700-550(Final Model 2 - 2 - check[20 epoche].png') # Model 2
    plt.show()

In [ ]:
loss, acc = model.evaluate(x_valid, y_valid, verbose = 0)
print("Validation Loss: %.2f" % (loss))
print("Validation Accuracy: %.3f" % (acc))
plot_history(history)

In [ ]:
results = model.predict(x_test)
results = np.argmax(results, axis = 1)

conv = {
    0: "A", 
    1: "B",
    2: "C",
    3: "D",
    4: "E",
    5: "F",
    6: "G",
    7: "H",
    8: "I",
    9: "J",
    10: "K",
    11: "L",
    12: "M",
    13: "N",
    14: "O",
    15: "P",
    16: "Q",
    17: "R",
    18: "S",
    19: "T",
    20: "U",
    21: "V",
    22: "W",
    23: "X",
    24: "Y",
    25: "Z",
    26: "del",
    27: "nothing",
    28: "space",
  }
y_pred = [conv[i] for i in results]
print("Prediction done:")

In [ ]:
print(classification_report(y_test, results))

In [ ]:
plt.figure(figsize = (12,12))
sns.heatmap(confusion_matrix(y_test, results))
plt.show()

In [ ]:
cm = confusion_matrix(y_test, results)
ConfusionMatrixDisplay.from_predictions(y_test, results, cmap="Blues")
# print(cm)

# Model Save

In [ ]:
model.save('/content/drive/MyDrive/VisionProject/Models/ASL.h5')
print("Model saved successfully")

# Model 1 Save

In [ ]:
model.save('/content/drive/MyDrive/VisionProject/Models/ASL1.h5')
print("Model saved successfully")

# Model 2 Save

In [ ]:
model.save('/content/drive/MyDrive/VisionProject/Models/ASL2.h5')
print("Model saved successfully")

# Model Load


In [ ]:
model = load_model('/content/drive/MyDrive/VisionProject/Models/ASL.h5')
model.load_weights('/content/drive/MyDrive/VisionProject/Weights/weights_best.hdf5')
print("Model restore with best weights")

# Model 1 Load

In [ ]:
model = load_model('/content/drive/MyDrive/VisionProject/Models/ASL1.h5')
model.load_weights('/content/drive/MyDrive/VisionProject/Weights/weights_best1.hdf5')
print("Model restore with best weights")

# Model 2 Load

In [ ]:
model = load_model('/content/drive/MyDrive/VisionProject/Models/ASL2.h5')
model.load_weights('/content/drive/MyDrive/VisionProject/Weights/weights_best2.hdf5')
print("Model restore with best weights")